<h1 style="text-align: center; font-size: 60px;">PREDICT TICKET CANCELLATION PROJECT</h1>
<h3 style="text-align: center; font-size: 20px;">(submitted on 3rd Oct 2025)</h3>

In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [162]:
df = pd.read_excel("TrainTicketCancellations.xlsx", sheet_name=1)
print(df.shape)

(60066, 13)


In [163]:
df.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,Untoward Incidents,State of Travellers,Weekday of Travel,Cancelled Or Not
0,Trivandrum,Ernakulam,70,July,7,1,NaN,Jan Satabdi,Second AC 3 Tier,NaN,Kerala,Tuesday,False
1,Panvel,Mookambika Road,51,July,1,2,NaN,Jan Satabdi,Second AC 2 Tier,NaN,Tamilnadu,Saturday,True
2,Madgaon,Kumta,40,February,3,0,Christmas,Express,Second Ordinary,NaN,Karnataka,Monday,True
3,Bhatkal,Vasai Road,38,February,5,3,Ramzan,Garib Rath,Second AC Economy,Heavy Rain,Gujarat,Monday,False
4,Trivandrum,Mangalore,92,October,6,2,Vishu,Rajadhani,First AC Coupe,NaN,Karnataka,Wedensday,False


In [164]:
df.isna().sum()

Starting Point                          0
Destination                             0
Booked Before Days                      0
Month of Travel                         0
Number of travellers                    0
Number of previous cancellations        0
Festival                            15140
Class of Train                          0
Class of Ticket                         0
Untoward Incidents                  49498
State of Travellers                     0
Weekday of Travel                       0
Cancelled Or Not                        0
dtype: int64

#### Removing rows where Starting Point = Destination.

In [165]:
df1 = df[df['Starting Point']!=df['Destination']]

In [166]:
df1[df1['Starting Point']==df1['Destination']]

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,Untoward Incidents,State of Travellers,Weekday of Travel,Cancelled Or Not


In [167]:
df1.describe()

,Booked Before Days,Number of travellers,Number of previous cancellations
count,57831.000000,57831.00000,57831.000000
mean,60.500199,4.00268,1.490844
std,34.578403,2.00470,1.114032
min,1.000000,1.00000,0.000000
25%,31.000000,2.00000,0.000000
50%,61.000000,4.00000,1.000000
75%,90.000000,6.00000,2.000000
max,120.000000,7.00000,3.000000


### Feature Engineering

In [170]:
# Load future bookings
future_df = pd.read_excel('FutureBookings.xlsx', sheet_name=1)

In [258]:
# Adding relevant columns

# 1. Lead time group
def lead_time_group(days):
    if days <= 3:
        return 'Very Short'
    elif days <= 14:
        return 'Short'
    elif days <= 30:
        return 'Medium'
    else:
        return 'Long'

df1['lead_time_group'] = df['Booked Before Days'].apply(lead_time_group)
future_df['lead_time_group'] = future_df['Booked Before Days'].apply(lead_time_group)

In [172]:
df1.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,Untoward Incidents,State of Travellers,Weekday of Travel,Cancelled Or Not,lead_time_group
0,Trivandrum,Ernakulam,70,July,7,1,NaN,Jan Satabdi,Second AC 3 Tier,NaN,Kerala,Tuesday,False,Long
1,Panvel,Mookambika Road,51,July,1,2,NaN,Jan Satabdi,Second AC 2 Tier,NaN,Tamilnadu,Saturday,True,Long
2,Madgaon,Kumta,40,February,3,0,Christmas,Express,Second Ordinary,NaN,Karnataka,Monday,True,Long
3,Bhatkal,Vasai Road,38,February,5,3,Ramzan,Garib Rath,Second AC Economy,Heavy Rain,Gujarat,Monday,False,Long
4,Trivandrum,Mangalore,92,October,6,2,Vishu,Rajadhani,First AC Coupe,NaN,Karnataka,Wedensday,False,Long


In [173]:
future_df.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,State of Travellers,Weekday of Travel,lead_time_group
0,Chiplun,Mookambika Road,19,October,3,1,Christmas,Jan Satabdi,First AC Coupe,Tamilnadu,Thursday,Medium
1,Madgaon,Panvel,55,January,4,1,NaN,Rajadhani,Second AC 2 Tier,Goa,Wedensday,Long
2,Kannur,Thrissur,22,December,4,1,NaN,Garib Rath,First AC Coupe,Gujarat,Saturday,Medium
3,Kozhikkode,Thivim,52,November,6,0,Pongal,Express,First AC,Kerala,Saturday,Long
4,Alappuzha,Bhatkal,19,December,7,3,NaN,Jan Satabdi,Second AC 3 Tier,Tamilnadu,Friday,Medium


In [174]:
# 2. Month/season
def travel_season(month):
    if month in ['January', 'February', 'March']:
        return 'OffPeak'
    elif month in ['April', 'May', 'June']:
        return 'Summer'
    elif month in ['July', 'August', 'September']:
        return 'Monsoon'
    else:
        return 'FestivalPeak'

df1['travel_season'] = df['Month of Travel'].apply(travel_season)
future_df['travel_season'] = future_df['Month of Travel'].apply(travel_season)

C:\Users\Pranav S P\AppData\Local\Temp\ipykernel_21064\1442981124.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['travel_season'] = df['Month of Travel'].apply(travel_season)


In [175]:
df1.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,Untoward Incidents,State of Travellers,Weekday of Travel,Cancelled Or Not,lead_time_group,travel_season
0,Trivandrum,Ernakulam,70,July,7,1,NaN,Jan Satabdi,Second AC 3 Tier,NaN,Kerala,Tuesday,False,Long,Monsoon
1,Panvel,Mookambika Road,51,July,1,2,NaN,Jan Satabdi,Second AC 2 Tier,NaN,Tamilnadu,Saturday,True,Long,Monsoon
2,Madgaon,Kumta,40,February,3,0,Christmas,Express,Second Ordinary,NaN,Karnataka,Monday,True,Long,OffPeak
3,Bhatkal,Vasai Road,38,February,5,3,Ramzan,Garib Rath,Second AC Economy,Heavy Rain,Gujarat,Monday,False,Long,OffPeak
4,Trivandrum,Mangalore,92,October,6,2,Vishu,Rajadhani,First AC Coupe,NaN,Karnataka,Wedensday,False,Long,FestivalPeak


In [176]:
future_df.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,State of Travellers,Weekday of Travel,lead_time_group,travel_season
0,Chiplun,Mookambika Road,19,October,3,1,Christmas,Jan Satabdi,First AC Coupe,Tamilnadu,Thursday,Medium,FestivalPeak
1,Madgaon,Panvel,55,January,4,1,NaN,Rajadhani,Second AC 2 Tier,Goa,Wedensday,Long,OffPeak
2,Kannur,Thrissur,22,December,4,1,NaN,Garib Rath,First AC Coupe,Gujarat,Saturday,Medium,FestivalPeak
3,Kozhikkode,Thivim,52,November,6,0,Pongal,Express,First AC,Kerala,Saturday,Long,FestivalPeak
4,Alappuzha,Bhatkal,19,December,7,3,NaN,Jan Satabdi,Second AC 3 Tier,Tamilnadu,Friday,Medium,FestivalPeak


In [177]:
# 3. Traveler group
def traveller_group(n):
    if n == 1:
        return 'Solo'
    elif n <= 4:
        return 'Small'
    else:
        return 'Large'

df1['traveller_group'] = df['Number of travellers'].apply(traveller_group)
future_df['traveller_group'] = future_df['Number of travellers'].apply(traveller_group)

C:\Users\Pranav S P\AppData\Local\Temp\ipykernel_21064\3797477796.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['traveller_group'] = df['Number of travellers'].apply(traveller_group)


In [178]:
df1.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,Untoward Incidents,State of Travellers,Weekday of Travel,Cancelled Or Not,lead_time_group,travel_season,traveller_group
0,Trivandrum,Ernakulam,70,July,7,1,NaN,Jan Satabdi,Second AC 3 Tier,NaN,Kerala,Tuesday,False,Long,Monsoon,Large
1,Panvel,Mookambika Road,51,July,1,2,NaN,Jan Satabdi,Second AC 2 Tier,NaN,Tamilnadu,Saturday,True,Long,Monsoon,Solo
2,Madgaon,Kumta,40,February,3,0,Christmas,Express,Second Ordinary,NaN,Karnataka,Monday,True,Long,OffPeak,Small
3,Bhatkal,Vasai Road,38,February,5,3,Ramzan,Garib Rath,Second AC Economy,Heavy Rain,Gujarat,Monday,False,Long,OffPeak,Large
4,Trivandrum,Mangalore,92,October,6,2,Vishu,Rajadhani,First AC Coupe,NaN,Karnataka,Wedensday,False,Long,FestivalPeak,Large


#### Featured out lead_time_groups, travel_season & traveller_group.
#### next:  OneHotEncoding Weekdays & other categorical features.

In [179]:
from sklearn.preprocessing import OneHotEncoder

# Initialize encoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # handle_unknown='ignore' avoids errors with new weekdays

# Fit encoder on historical weekday column
ohe.fit(df1[['Weekday of Travel']])

# Transform and convert to DataFrame
weekday_encoded = pd.DataFrame(ohe.transform(df1[['Weekday of Travel']]), 
                               columns=ohe.get_feature_names_out(['Weekday of Travel']))

In [180]:
# Add to original df
df1 = pd.concat([df1.reset_index(drop=True), weekday_encoded], axis=1)
df1.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,Untoward Incidents,...,lead_time_group,travel_season,traveller_group,Weekday of Travel_Friday,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday
0,Trivandrum,Ernakulam,70,July,7,1,NaN,Jan Satabdi,Second AC 3 Tier,NaN,...,Long,Monsoon,Large,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Panvel,Mookambika Road,51,July,1,2,NaN,Jan Satabdi,Second AC 2 Tier,NaN,...,Long,Monsoon,Solo,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Madgaon,Kumta,40,February,3,0,Christmas,Express,Second Ordinary,NaN,...,Long,OffPeak,Small,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Bhatkal,Vasai Road,38,February,5,3,Ramzan,Garib Rath,Second AC Economy,Heavy Rain,...,Long,OffPeak,Large,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Trivandrum,Mangalore,92,October,6,2,Vishu,Rajadhani,First AC Coupe,NaN,...,Long,FestivalPeak,Large,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [181]:
# Drop Weekday Column
df1.drop('Weekday of Travel', axis=1, inplace=True)

In [182]:
df1.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,Untoward Incidents,...,lead_time_group,travel_season,traveller_group,Weekday of Travel_Friday,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday
0,Trivandrum,Ernakulam,70,July,7,1,NaN,Jan Satabdi,Second AC 3 Tier,NaN,...,Long,Monsoon,Large,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Panvel,Mookambika Road,51,July,1,2,NaN,Jan Satabdi,Second AC 2 Tier,NaN,...,Long,Monsoon,Solo,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Madgaon,Kumta,40,February,3,0,Christmas,Express,Second Ordinary,NaN,...,Long,OffPeak,Small,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Bhatkal,Vasai Road,38,February,5,3,Ramzan,Garib Rath,Second AC Economy,Heavy Rain,...,Long,OffPeak,Large,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Trivandrum,Mangalore,92,October,6,2,Vishu,Rajadhani,First AC Coupe,NaN,...,Long,FestivalPeak,Large,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Similarly for future_df

In [183]:
# Transform future data using same encoder
weekday_encoded_future = pd.DataFrame(ohe.transform(future_df[['Weekday of Travel']]),
                                      columns=ohe.get_feature_names_out(['Weekday of Travel']))

# Add to future df
future_df = pd.concat([future_df.reset_index(drop=True), weekday_encoded_future], axis=1)
future_df.drop('Weekday of Travel', axis=1, inplace=True)

In [184]:
future_df.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,State of Travellers,lead_time_group,travel_season,traveller_group,Weekday of Travel_Friday,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday
0,Chiplun,Mookambika Road,19,October,3,1,Christmas,Jan Satabdi,First AC Coupe,Tamilnadu,Medium,FestivalPeak,Small,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Madgaon,Panvel,55,January,4,1,NaN,Rajadhani,Second AC 2 Tier,Goa,Long,OffPeak,Small,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Kannur,Thrissur,22,December,4,1,NaN,Garib Rath,First AC Coupe,Gujarat,Medium,FestivalPeak,Small,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Kozhikkode,Thivim,52,November,6,0,Pongal,Express,First AC,Kerala,Long,FestivalPeak,Large,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Alappuzha,Bhatkal,19,December,7,3,NaN,Jan Satabdi,Second AC 3 Tier,Tamilnadu,Medium,FestivalPeak,Large,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
df1 = df1.drop('Untoward Incidents', axis = 1)
df1.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,State of Travellers,...,lead_time_group,travel_season,traveller_group,Weekday of Travel_Friday,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday
0,Trivandrum,Ernakulam,70,July,7,1,NaN,Jan Satabdi,Second AC 3 Tier,Kerala,...,Long,Monsoon,Large,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Panvel,Mookambika Road,51,July,1,2,NaN,Jan Satabdi,Second AC 2 Tier,Tamilnadu,...,Long,Monsoon,Solo,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Madgaon,Kumta,40,February,3,0,Christmas,Express,Second Ordinary,Karnataka,...,Long,OffPeak,Small,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Bhatkal,Vasai Road,38,February,5,3,Ramzan,Garib Rath,Second AC Economy,Gujarat,...,Long,OffPeak,Large,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Trivandrum,Mangalore,92,October,6,2,Vishu,Rajadhani,First AC Coupe,Karnataka,...,Long,FestivalPeak,Large,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [186]:
df1.columns

Index(['Starting Point', 'Destination', 'Booked Before Days',
       'Month of Travel', 'Number of travellers',
       'Number of previous cancellations', 'Festival', 'Class of Train',
       'Class of Ticket', 'State of Travellers', 'Cancelled Or Not',
       'lead_time_group', 'travel_season', 'traveller_group',
       'Weekday of Travel_Friday', 'Weekday of Travel_Monday',
       'Weekday of Travel_Saturday', 'Weekday of Travel_Sunday',
       'Weekday of Travel_Thursday', 'Weekday of Travel_Tuesday',
       'Weekday of Travel_Wedensday'],
      dtype='object')

In [190]:
# Remove unecessary columns from historical as well future df.
df1 = df1.drop(columns = ['Month of Travel', 'Festival'])
future_df = future_df.drop(columns = ['Month of Travel', 'Festival'])

In [193]:
# To avoid dummy_trap

df1 = df1.drop(columns = ['Weekday of Travel_Friday'])
future_df = future_df.drop(columns = ['Weekday of Travel_Friday'])

In [197]:
future_df.columns

Index(['Starting Point', 'Destination', 'Booked Before Days',
       'Number of travellers', 'Number of previous cancellations',
       'Class of Train', 'Class of Ticket', 'State of Travellers',
       'lead_time_group', 'travel_season', 'traveller_group',
       'Weekday of Travel_Monday', 'Weekday of Travel_Saturday',
       'Weekday of Travel_Sunday', 'Weekday of Travel_Thursday',
       'Weekday of Travel_Tuesday', 'Weekday of Travel_Wedensday'],
      dtype='object')

In [198]:
df1.columns

Index(['Starting Point', 'Destination', 'Booked Before Days',
       'Number of travellers', 'Number of previous cancellations',
       'Class of Train', 'Class of Ticket', 'State of Travellers',
       'Cancelled Or Not', 'lead_time_group', 'travel_season',
       'traveller_group', 'Weekday of Travel_Monday',
       'Weekday of Travel_Saturday', 'Weekday of Travel_Sunday',
       'Weekday of Travel_Thursday', 'Weekday of Travel_Tuesday',
       'Weekday of Travel_Wedensday'],
      dtype='object')

### Categorical Encoding

In [211]:
from sklearn.compose import ColumnTransformer

# Select categorical features to one-hot
categorical_features = ['Starting Point', 'Destination', 'lead_time_group', 
                        'travel_season', 'traveller_group', 'Class of Train', 'Class of Ticket', 'State of Travellers']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ], remainder='passthrough'
)

## Splitting Data and Modelling

In [212]:
df1.head(2)

,Starting Point,Destination,Booked Before Days,Number of travellers,Number of previous cancellations,Class of Train,Class of Ticket,State of Travellers,Cancelled Or Not,lead_time_group,travel_season,traveller_group,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday
0,Trivandrum,Ernakulam,70,7,1,Jan Satabdi,Second AC 3 Tier,Kerala,False,Long,Monsoon,Large,0.0,0.0,0.0,0.0,1.0,0.0
1,Panvel,Mookambika Road,51,1,2,Jan Satabdi,Second AC 2 Tier,Tamilnadu,True,Long,Monsoon,Solo,0.0,1.0,0.0,0.0,0.0,0.0


In [213]:
from sklearn.model_selection import train_test_split

X = df1.drop('Cancelled Or Not', axis = 1)
y = df1['Cancelled Or Not']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## Using different Models through Pipeline

### Pipeline

In [259]:
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score

In [219]:
pipelines = {
    'Logistic Regression': Pipeline([('preprocessor', preprocessor),
                                     ('classifier', LogisticRegression(max_iter=1000))]),
    
    'Naive Bayes': Pipeline([('preprocessor', preprocessor),
                             ('classifier', GaussianNB())]),
    
    'Random Forest': Pipeline([('preprocessor', preprocessor),
                               ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))]),
}


## Train & Evaluate Classifiers Step by Step

In [227]:
results = {}

for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    results[name] = f1
    
    print(f"\n{name}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("F1-score: ", f1)
    print("Accuarcy: ", accuracy_score(y_test, y_pred))


Logistic Regression
Confusion Matrix:
 [[9251    0]
 [2316    0]]
Classification Report:
               precision    recall  f1-score   support

       False       0.80      1.00      0.89      9251
        True       0.00      0.00      0.00      2316

    accuracy                           0.80     11567
   macro avg       0.40      0.50      0.44     11567
weighted avg       0.64      0.80      0.71     11567

F1-score:  0.0
Accuarcy:  0.7997752226160629


C:\Users\Pranav S P\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Pranav S P\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Pranav S P\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"


Naive Bayes
Confusion Matrix:
 [[7712 1539]
 [1930  386]]
Classification Report:
               precision    recall  f1-score   support

       False       0.80      0.83      0.82      9251
        True       0.20      0.17      0.18      2316

    accuracy                           0.70     11567
   macro avg       0.50      0.50      0.50     11567
weighted avg       0.68      0.70      0.69     11567

F1-score:  0.18203253949540202
Accuarcy:  0.7000950981239734

Random Forest
Confusion Matrix:
 [[9250    1]
 [2316    0]]
Classification Report:
               precision    recall  f1-score   support

       False       0.80      1.00      0.89      9251
        True       0.00      0.00      0.00      2316

    accuracy                           0.80     11567
   macro avg       0.40      0.50      0.44     11567
weighted avg       0.64      0.80      0.71     11567

F1-score:  0.0
Accuarcy:  0.7996887697760872


In [228]:
best_model_name = max(results, key=results.get)
print("Best classifier based on F1-score:", best_model_name)

Best classifier based on F1-score: Naive Bayes


## Using Naive Bayes to predict Future Bookings Cancellation.

In [234]:
future_df.head(2)

,Starting Point,Destination,Booked Before Days,Number of travellers,Number of previous cancellations,Class of Train,Class of Ticket,State of Travellers,lead_time_group,travel_season,traveller_group,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday
0,Chiplun,Mookambika Road,19,3,1,Jan Satabdi,First AC Coupe,Tamilnadu,Medium,FestivalPeak,Small,0.0,0.0,0.0,1.0,0.0,0.0
1,Madgaon,Panvel,55,4,1,Rajadhani,Second AC 2 Tier,Goa,Long,OffPeak,Small,0.0,0.0,0.0,0.0,0.0,1.0


In [235]:
nb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GaussianNB())
])

In [236]:
X = df1.drop('Cancelled Or Not', axis = 1)
y = df1['Cancelled Or Not']

nb_pipeline.fit(X, y)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [254]:
X_future = future_df
future_df['Cancelled or Not'] = nb_pipeline.predict(X_future).astype(int)
future_df['Probability_of_Cancellation'] = nb_pipeline.predict_proba(X_future)[:,1]

In [255]:
future_df.head()

,Starting Point,Destination,Booked Before Days,Number of travellers,Number of previous cancellations,Class of Train,Class of Ticket,State of Travellers,lead_time_group,travel_season,...,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday,predicted_label,predicted_prob,Cancelled or Not,Probability_of_Cancellation
0,Chiplun,Mookambika Road,19,3,1,Jan Satabdi,First AC Coupe,Tamilnadu,Medium,FestivalPeak,...,0.0,0.0,0.0,1.0,0.0,0.0,0,0.375031,0,0.375031
1,Madgaon,Panvel,55,4,1,Rajadhani,Second AC 2 Tier,Goa,Long,OffPeak,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0.690605,1,0.690605
2,Kannur,Thrissur,22,4,1,Garib Rath,First AC Coupe,Gujarat,Medium,FestivalPeak,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0.107644,0,0.107644
3,Kozhikkode,Thivim,52,6,0,Express,First AC,Kerala,Long,FestivalPeak,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0.310645,0,0.310645
4,Alappuzha,Bhatkal,19,7,3,Jan Satabdi,Second AC 3 Tier,Tamilnadu,Medium,FestivalPeak,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.403926,0,0.403926


# Prediction Completed.
## Now writing the predictions to a csv file.

In [256]:
# Save results
future_df.to_csv('FutureBookings_Predictions.csv', index=False)

In [257]:
data = pd.read_csv('FutureBookings_Predictions.csv')
data

,Starting Point,Destination,Booked Before Days,Number of travellers,Number of previous cancellations,Class of Train,Class of Ticket,State of Travellers,lead_time_group,travel_season,...,Weekday of Travel_Monday,Weekday of Travel_Saturday,Weekday of Travel_Sunday,Weekday of Travel_Thursday,Weekday of Travel_Tuesday,Weekday of Travel_Wedensday,predicted_label,predicted_prob,Cancelled or Not,Probability_of_Cancellation
0,Chiplun,Mookambika Road,19,3,1,Jan Satabdi,First AC Coupe,Tamilnadu,Medium,FestivalPeak,...,0.0,0.0,0.0,1.0,0.0,0.0,0,0.375031,0,0.375031
1,Madgaon,Panvel,55,4,1,Rajadhani,Second AC 2 Tier,Goa,Long,OffPeak,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0.690605,1,0.690605
2,Kannur,Thrissur,22,4,1,Garib Rath,First AC Coupe,Gujarat,Medium,FestivalPeak,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0.107644,0,0.107644
3,Kozhikkode,Thivim,52,6,0,Express,First AC,Kerala,Long,FestivalPeak,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0.310645,0,0.310645
4,Alappuzha,Bhatkal,19,7,3,Jan Satabdi,Second AC 3 Tier,Tamilnadu,Medium,FestivalPeak,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.403926,0,0.403926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10584,Bangalore,Mumbai CST,1,4,2,Superfast,Second AC 2 Tier,Goa,Very Short,FestivalPeak,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0.713148,1,0.713148
10585,Ernakulam,Ratnagiri,4,1,2,Rajadhani,Second Ordinary,Karnataka,Short,OffPeak,...,0.0,0.0,0.0,0.0,0.0,1.0,0,0.284182,0,0.284182
10586,Kannur,Vasai Road,29,6,0,Superfast,Second AC 2 Tier,Gujarat,Medium,OffPeak,...,0.0,0.0,1.0,0.0,0.0,0.0,0,0.050758,0,0.050758
10587,Ernakulam,Roha,23,6,1,Express,Second AC 3 Tier,Tamilnadu,Medium,OffPeak,...,0.0,0.0,1.0,0.0,0.0,0.0,0,0.138348,0,0.138348


<h1 style="text-align: center; font-size: 50px;">PROJECT COMPLETED</h1>